In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [238]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'

In [239]:
dataset = pd.read_csv('winequality-white.csv',delimiter=';')
x=dataset.iloc[:,:-1].values
y=dataset.iloc[:,-1].values
#y=y.reshape(-1,1)

In [240]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [241]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
x_train=mms.fit_transform(x_train)
x_test=mms.transform(x_test)

In [242]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y_train=le.fit_transform(y_train)
# y_test = le.transform(y_test)
#print(y_test)


In [243]:
#print(y_train)

In [244]:
x_train=torch.FloatTensor(x_train).to(device)
x_test=torch.FloatTensor(x_test).to(device)
y_train=torch.LongTensor(y_train).to(device)
y_test=torch.LongTensor(y_test).to(device)
y_train = torch.sub(y_train, 3)
y_test = torch.sub(y_test, 3)

In [245]:
loss_fn=nn.CrossEntropyLoss()
model=nn.Sequential(nn.Linear(len(x[0]),35),nn.ReLU(),
                    nn.Linear(35,52),nn.ReLU(),
                    nn.Linear(52,72),nn.ReLU(),
                    nn.Linear(72,52),nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(52,52),nn.ReLU(),
                    nn.Linear(52,42),nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(42,52),nn.ReLU(),
                    nn.Linear(52,52),nn.ReLU(),
                    nn.Linear(52,52),nn.ReLU(),
                    nn.Linear(52,7)).to(device)
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.02)

In [246]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [247]:
#print(model(x_train))

In [248]:
#print(torch.argmax(model(x_train),dim=1))

In [249]:
torch.manual_seed(42)

test_preds=list()
test_losses=list()

for epoch in range(10001):
    model.train()
    y_logits=model(x_train)
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
    loss=loss_fn(y_logits,y_train)
    acc=accuracy_fn(y_train,y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.inference_mode():
        test_logits = model(x_test)
        test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,y_pred=test_pred)
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%") 

Epoch: 0 | Loss: 1.93793, Acc: 3.57% | Test Loss: 1.79008, Test Acc: 44.08%
Epoch: 1000 | Loss: 0.96701, Acc: 59.19% | Test Loss: 1.06031, Test Acc: 56.53%
Epoch: 2000 | Loss: 0.94161, Acc: 59.72% | Test Loss: 1.09814, Test Acc: 57.24%
Epoch: 3000 | Loss: 0.92631, Acc: 60.82% | Test Loss: 1.09200, Test Acc: 55.20%
Epoch: 4000 | Loss: 0.90887, Acc: 60.97% | Test Loss: 1.11285, Test Acc: 56.02%
Epoch: 5000 | Loss: 0.94438, Acc: 60.59% | Test Loss: 1.12655, Test Acc: 57.14%
Epoch: 6000 | Loss: 0.90635, Acc: 61.77% | Test Loss: 1.16345, Test Acc: 55.20%
Epoch: 7000 | Loss: 0.94097, Acc: 60.41% | Test Loss: 1.08707, Test Acc: 55.61%
Epoch: 8000 | Loss: 0.99723, Acc: 57.96% | Test Loss: 1.11196, Test Acc: 55.00%
Epoch: 9000 | Loss: 1.01927, Acc: 56.36% | Test Loss: 1.05739, Test Acc: 55.00%
Epoch: 10000 | Loss: 0.97922, Acc: 57.68% | Test Loss: 1.04624, Test Acc: 55.31%
